In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hemochromatosis"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Hemochromatosis/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Hemochromatosis/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Hemochromatosis/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Hemochromatosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. List all subdirectories in the TCGA root directory
subdirectories = os.listdir(tcga_root_dir)
print(f"Available TCGA subdirectories: {subdirectories}")

# The target trait is Hemochromatosis
# Define key terms relevant to Hemochromatosis
key_terms = ["liver", "iron", "hemochromatosis", "hepatic", "metal", "LIHC"]

# Initialize variables for best match
best_match = None
best_match_score = 0
min_threshold = 1  # Require at least 1 matching term

# Convert trait to lowercase for case-insensitive matching
target_trait = trait.lower()  # "hemochromatosis"

# Search for relevant directories
for subdir in subdirectories:
    if not os.path.isdir(os.path.join(tcga_root_dir, subdir)) or subdir.startswith('.'):
        continue
        
    subdir_lower = subdir.lower()
    
    # Check for exact matches
    if target_trait in subdir_lower:
        best_match = subdir
        print(f"Found exact match: {subdir}")
        break
    
    # Calculate score based on key terms
    score = 0
    for term in key_terms:
        if term in subdir_lower:
            score += 1
    
    # Update best match if score is higher than current best
    if score > best_match_score and score >= min_threshold:
        best_match_score = score
        best_match = subdir
        print(f"Found potential match: {subdir} (score: {score})")

# If no match found, LIHC (Liver Cancer) is most relevant for Hemochromatosis
if not best_match and "TCGA_Liver_Cancer_(LIHC)" in subdirectories:
    best_match = "TCGA_Liver_Cancer_(LIHC)"
    print(f"Selected {best_match} as most relevant to Hemochromatosis which affects the liver")

# Handle the case where a match is found
if best_match:
    print(f"Selected directory: {best_match}")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, best_match)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and trait data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print(f"No suitable directory found for {trait}.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not best_match:
    print("Skipping this trait as no suitable data was found in TCGA.")


Available TCGA subdirectories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Canc


Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'adjacent_hepatic_tissue_inflammation_extent_type', 'age_at_initial_pathologic_diagnosis', 'albumin_result_lower_limit', 'albumin_result_specified_value', 'albumin_result_upper_limit', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'bilirubin_lower_limit', 'bilirubin_upper_limit', 'cancer_first_degree_relative', 'child_pugh_classification_grade', 'creatinine_lower_level', 'creatinine_upper_limit', 'creatinine_value_in_mg_dl', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'eastern_cancer_oncology_group', 'fetoprotein_outcome_lower_limit', 'fetoprotein_outcome_upper_limit', 'fetoprotein_outcome_value', 'fibrosis_ishak_sc

### Step 2: Find Candidate Demographic Features

In [3]:
# Examine the clinical data columns to identify candidate age and gender columns
candidate_age_cols = ["age_at_initial_pathologic_diagnosis", "days_to_birth"]
candidate_gender_cols = ["gender"]

# Let's get the clinical data file path first
lihc_dir = os.path.join(tcga_root_dir, "TCGA_Liver_Cancer_(LIHC)")
clinical_file_path, _ = tcga_get_relevant_filepaths(lihc_dir)

# Load the clinical data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview the age columns
age_preview = {}
if candidate_age_cols:
    age_data = clinical_df[candidate_age_cols]
    age_preview = preview_df(age_data)
    print("Age columns preview:")
    print(age_preview)

# Extract and preview the gender columns
gender_preview = {}
if candidate_gender_cols:
    gender_data = clinical_df[candidate_gender_cols]
    gender_preview = preview_df(gender_data)
    print("Gender columns preview:")
    print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [nan, 58.0, 51.0, 55.0, 54.0], 'days_to_birth': [nan, -21318.0, -18768.0, -20187.0, -20011.0]}
Gender columns preview:
{'gender': ['MALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Select appropriate columns for age and gender based on the provided previews

# For age:
# Both 'age_at_initial_pathologic_diagnosis' and 'days_to_birth' contain meaningful data
# 'age_at_initial_pathologic_diagnosis' directly gives age in years, which is more intuitive
# 'days_to_birth' gives negative days since birth, which would need conversion
age_col = 'age_at_initial_pathologic_diagnosis'

# For gender:
# Only one column is available and it has meaningful values (MALE/FEMALE)
gender_col = 'gender'

# Print chosen columns
print(f"Chosen age column: {age_col}")
print(f"Age column preview: {{'age_at_initial_pathologic_diagnosis': [nan, 58.0, 51.0, 55.0, 54.0]}}")

print(f"Chosen gender column: {gender_col}")
print(f"Gender column preview: {{'gender': ['MALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']}}")


Chosen age column: age_at_initial_pathologic_diagnosis
Age column preview: {'age_at_initial_pathologic_diagnosis': [nan, 58.0, 51.0, 55.0, 54.0]}
Chosen gender column: gender
Gender column preview: {'gender': ['MALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']}


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Create clinical features dataframe with trait (Canavan Disease) using patient IDs
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Step 2: Normalize gene symbols in the gene expression data
# The gene symbols in TCGA genetic data are already standardized, but we'll normalize them for consistency
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Gene expression data shape after normalization: {normalized_gene_df.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_t = normalized_gene_df.T
# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")

# Verify common indices between clinical and genetic data
clinical_indices = set(clinical_features.index)
genetic_indices = set(genetic_df_t.index)
common_indices = clinical_indices.intersection(genetic_indices)
print(f"Number of samples in clinical data: {len(clinical_indices)}")
print(f"Number of samples in genetic data: {len(genetic_indices)}")
print(f"Number of common samples: {len(common_indices)}")

# Link the data by using the common indices
linked_data = pd.concat([clinical_features.loc[list(common_indices)], genetic_df_t.loc[list(common_indices)]], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Step 5: Determine whether the trait and demographic features are severely biased
trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)

# Step 6: Conduct final quality validation and save information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data,
    note=f"Dataset contains TCGA glioma and brain tumor samples with gene expression and clinical information for {trait}."
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable based on validation criteria. Data not saved.")

print("Preprocessing completed.")

Normalized gene expression data saved to ../../output/preprocess/Hemochromatosis/gene_data/TCGA.csv
Gene expression data shape after normalization: (19848, 423)
Clinical data saved to ../../output/preprocess/Hemochromatosis/clinical_data/TCGA.csv
Clinical data shape: (438, 3)
Number of samples in clinical data: 438
Number of samples in genetic data: 423
Number of common samples: 423
Linked data shape: (423, 19851)


Data shape after handling missing values: (423, 19851)
For the feature 'Hemochromatosis', the least common label is '0' with 50 occurrences. This represents 11.82% of the dataset.
The distribution of the feature 'Hemochromatosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 52.0
  50% (Median): 62.0
  75%: 69.0
Min: 16.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 143 occurrences. This represents 33.81% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Hemochromatosis/TCGA.csv
Preprocessing completed.
